In [1]:
import torch
import torch.nn as nn
import torch.optim as optim

# import spacy

import random
import math
import time

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [2]:
data_dir = './'
proj_dir = './'

In [0]:
!cp '/content/drive/My Drive/sdml/HW2/model/Decoder.py' .
!cp '/content/drive/My Drive/sdml/HW2/model/Encoder.py' .
!cp '/content/drive/My Drive/sdml/HW2/model/Seq2Seq.py' .

In [3]:
OOV_list = []
OOV=0
class Vocabulary(object):

    def __init__(self):
        self.word2idx = {'<sos>': 0, '<eos>': 1, '<pad>': 2, '<unk>': 3}
        self.idx2word = {0: '<sos>', 1: '<eos>', 2: '<pad>', 3: '<unk>'}
        self.num_words = 4
#         self.max_length = 0
#         self.sent_list = []

    def build_vocab(self, data_path):
        """Construct the relation between words and indices"""
        with open(data_path, 'r', encoding='utf-8') as dataset:
            for words in dataset:
                words = words.strip('\n').split(' ')

#                 self.sent_list.append(words)
#                 if self.max_length < len(words):
#                     self.max_length = len(words)

#                 chars = self.split_sequence(word)
                for word in words:
                    word = word.lower()
                    if word not in self.word2idx:
                        self.word2idx[word] = self.num_words
                        self.idx2word[self.num_words] = word
                        self.num_words += 1

    def sequence_to_indices(self, sequence, add_eos=False, add_sos=False):
        """Transform a char sequence to index sequence
            :param sequence: a string composed with chars
            :param add_eos: if true, add the <EOS> tag at the end of given sentence
            :param add_sos: if true, add the <SOS> tag at the beginning of given sentence
        """
        index_sequence = [self.word2idx['<sos>']] if add_sos else []

        for word in sequence:
            word = word.lower()
            if word not in self.word2idx:
                OOV+=1
                OOV_list.append(word)
                print("!!!!")
                index_sequence.append((self.word2idx['<unk>']))
            else:
                index_sequence.append(self.word2idx[word])

        if add_eos:
            index_sequence.append(self.word2idx['<eos>'])

        return index_sequence

    def indices_to_sequence(self, indices):
        """Transform a list of indices
            :param indices: a list
        """
        sequence = []
        for idx in indices:
            word = self.idx2word[idx]
            if word == "<eos>":
                sequence.append(word)
                break
            else:
                sequence.append(word)
        return ' '.join(sequence)

    def __str__(self):
        str = "Vocab information:\n"
        for idx, word in self.idx2word.items():
            str += "word: %s Index: %d\n" % (word, idx)
        return str

In [4]:
from torch.utils.data import DataLoader, Dataset, SequentialSampler, RandomSampler
import torch.nn.utils.rnn as rnn_utils
class MyData(Dataset):
    def __init__(self, path,vocab=None):
        self.src_indices_seq = []
        self.trg_indices_seq = []
        if vocab == None:
            self.vocab = Vocabulary()
            self.vocab.build_vocab(path)
        else:
            self.vocab = vocab
        self.PAD_ID = self.vocab.word2idx["<pad>"]
        self.SOS_ID = self.vocab.word2idx["<sos>"]
        self.vocab_size = self.vocab.num_words
        self.max_length = -1
        self.raw_data = []
#         self.max_length = self.vocab.max_length
        file=open(path, 'r', encoding='utf-8')
        for words in file:
            words = [i.lower() for i in words.strip('\n').split(' ')]
            self.raw_data.append(words)
            if self.max_length < len(words):
                self.max_length = len(words)
            
        for sent in self.raw_data:
            indices_seq = self.vocab.sequence_to_indices(sent, add_eos=False)
            # input and target are the same in auto-encoder
            self.src_indices_seq.append(torch.tensor(indices_seq))
            self.trg_indices_seq.append(torch.tensor(indices_seq))
        self.src_indices_seq = rnn_utils.pad_sequence(self.src_indices_seq, batch_first=True, padding_value=self.PAD_ID)
        self.trg_indices_seq = rnn_utils.pad_sequence(self.trg_indices_seq, batch_first=True, padding_value=self.PAD_ID)
        print("## J: Total examples: %d, unique words:%d, Max seq length: %d"%(len(self.src_indices_seq),self.vocab_size,self.max_length))
    # def collate_fn(data):
    #     data = rnn_utils.pad_sequence(data, batch_first=True, padding_value=0)
    #     return data
    def __len__(self):
        return len(self.src_indices_seq)

    def __getitem__(self, idx):
        return torch.tensor(self.src_indices_seq[idx]), torch.tensor(self.trg_indices_seq[idx])


In [5]:
train_set = MyData(data_dir+'train.txt')
test_set = MyData(data_dir+'200_vocab_dataset.txt',vocab=train_set.vocab)

## J: Total examples: 382785, unique words:19738, Max seq length: 13
## J: Total examples: 20425, unique words:19738, Max seq length: 16


In [0]:
# data_loader = DataLoader(train_set, batch_size=4, shuffle=True)
# batch_x = iter(data_loader).next()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:28: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).


In [7]:
batch_x[0].shape
# train_set.src_indices_seq.size()

NameError: name 'batch_x' is not defined

In [6]:
SEED = 1234

random.seed(SEED)
torch.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

In [7]:
device = torch.device('cuda')

In [8]:
class Encoder(nn.Module):
    def __init__(self, input_dim, emb_dim, hid_dim, n_layers, dropout):
        super().__init__()
        
        self.hid_dim = hid_dim
        self.n_layers = n_layers
        
#         self.embedding = nn.Embedding(input_dim, emb_dim)
        
        # self.rnn = nn.LSTM(emb_dim, hid_dim, n_layers, dropout = dropout)
        self.gru = nn.GRU(emb_dim, hid_dim, n_layers, dropout = dropout,batch_first=True)
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, src):
        
        #src = [src sent len, batch size]
        
#         embedded = self.dropout(self.embedding(src))
        embedded = self.dropout(src)
        
        #embedded = [src sent len, batch size, emb dim]
        
        # outputs, (hidden, cell) = self.rnn(embedded)
        outputs, hidden = self.gru(embedded)
        
        #outputs = [src sent len, batch size, hid dim * n directions]
        #hidden = [n layers * n directions, batch size, hid dim]
        #cell = [n layers * n directions, batch size, hid dim]
        
        #outputs are always from the top hidden layer
        
        return hidden

In [9]:
class Decoder(nn.Module):
    def __init__(self, output_dim, emb_dim, hid_dim, n_layers, dropout):
        super().__init__()
        
        self.output_dim = output_dim
        self.hid_dim = hid_dim
        self.n_layers = n_layers
        
#         self.embedding = nn.Embedding(output_dim, emb_dim)
        
        # self.rnn = nn.LSTM(emb_dim, hid_dim, n_layers, dropout = dropout)
        self.gru = nn.GRU(emb_dim, hid_dim, n_layers, dropout = dropout,batch_first=True)
        
        self.out = nn.Linear(hid_dim, output_dim)
        
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, input, hidden):
        
        #input = [batch size]
        #hidden = [n layers * n directions, batch size, hid dim]
        #cell = [n layers * n directions, batch size, hid dim]
        
        #n directions in the decoder will both always be 1, therefore:
        #hidden = [n layers, batch size, hid dim]
        #context = [n layers, batch size, hid dim]
        
#         input = input.unsqueeze(1)
        
#         #input = [1, batch size]
        
#         embedded = self.dropout(self.embedding(input))
        embedded = self.dropout(input)
        
        #embedded = [1, batch size, emb dim]
                
        # output, (hidden, cell) = self.rnn(embedded, (hidden, cell))
        output, hidden = self.gru(embedded,hidden)
        
        #output = [sent len, batch size, hid dim * n directions]
        #hidden = [n layers * n directions, batch size, hid dim]
        #cell = [n layers * n directions, batch size, hid dim]
        
        #sent len and n directions will always be 1 in the decoder, therefore:
        #output = [1, batch size, hid dim]
        #hidden = [n layers, batch size, hid dim]
        #cell = [n layers, batch size, hid dim]
        
        prediction = self.out(output.squeeze(1))
        
        #prediction = [batch size, output dim]
        
        return prediction, hidden

In [10]:
class Seq2Seq(nn.Module):
    def __init__(self, encoder, decoder, device,output_dim, emb_dim):
        super().__init__()
        self.embedding = nn.Embedding(output_dim, emb_dim)
        self.encoder = encoder
        self.decoder = decoder
        self.device = device
        
        assert encoder.hid_dim == decoder.hid_dim, \
            "Hidden dimensions of encoder and decoder must be equal!"
        assert encoder.n_layers == decoder.n_layers, \
            "Encoder and decoder must have equal number of layers!"
        
    def forward(self, src, trg, teacher_forcing_ratio = 0.5):
        
        #src = [src sent len, batch size]
        #trg = [trg sent len, batch size]
        #teacher_forcing_ratio is probability to use teacher forcing
        #e.g. if teacher_forcing_ratio is 0.75 we use ground-truth inputs 75% of the time
        
        batch_size = trg.shape[0]
        max_len = trg.shape[1]
        trg_vocab_size = self.decoder.output_dim
        
        #tensor to store decoder outputs
#         outputs = torch.zeros(max_len, batch_size, trg_vocab_size).to(self.device)
        outputs = torch.zeros(batch_size, max_len, trg_vocab_size).to(self.device)
        outputs_idx = torch.zeros(batch_size, max_len-1)

        #last hidden state of the encoder is used as the initial hidden state of the decoder
#         hidden = self.encoder(src)
        embedded = self.embedding(src)
        hidden = self.encoder(embedded)
        
        #first input to the decoder is the <sos> tokens
        input = trg[:,0]
        
        for t in range(1, max_len):
            
            #insert input token embedding, previous hidden and previous cell states
            #receive output tensor (predictions) and new hidden and cell states
#             output, hidden= self.decoder(input, hidden)
            input = input.unsqueeze(1)
            embedded = self.embedding(input)
            output, hidden= self.decoder(embedded, hidden)
            
            #place predictions in a tensor holding predictions for each token
            outputs[:,t,:] = output
            
            #decide if we are going to use teacher forcing or not
            teacher_force = random.random() < teacher_forcing_ratio
            
            #get the highest predicted token from our predictions
            top1 = output.argmax(1)
            outputs_idx[:,t-1] = top1
            
            #if teacher forcing, use actual next token as next input
            #if not, use predicted token
            input = trg[:,t] if teacher_force else top1
        
        return outputs,outputs_idx

In [11]:
INPUT_DIM = train_set.vocab_size
OUTPUT_DIM = train_set.vocab_size
ENC_EMB_DIM = 256
DEC_EMB_DIM = 256
HID_DIM = 512
N_LAYERS = 1
ENC_DROPOUT = 0
DEC_DROPOUT = 0

enc = Encoder(INPUT_DIM, ENC_EMB_DIM, HID_DIM, N_LAYERS, ENC_DROPOUT)
dec = Decoder(OUTPUT_DIM, DEC_EMB_DIM, HID_DIM, N_LAYERS, DEC_DROPOUT)

model = Seq2Seq(enc, dec, device,INPUT_DIM, ENC_EMB_DIM).to(device)

In [12]:
def init_weights(m):
    for name, param in m.named_parameters():
        nn.init.uniform_(param.data, -0.08, 0.08)
        
model.apply(init_weights)

Seq2Seq(
  (embedding): Embedding(19738, 256)
  (encoder): Encoder(
    (gru): GRU(256, 512, batch_first=True)
    (dropout): Dropout(p=0, inplace=False)
  )
  (decoder): Decoder(
    (gru): GRU(256, 512, batch_first=True)
    (out): Linear(in_features=512, out_features=19738, bias=True)
    (dropout): Dropout(p=0, inplace=False)
  )
)

In [13]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')

The model has 17,543,962 trainable parameters


In [14]:
optimizer = optim.Adam(model.parameters())

In [15]:
criterion = nn.CrossEntropyLoss(ignore_index = train_set.PAD_ID)

In [16]:
from tqdm import tqdm, trange
def train(model, optimizer, criterion, clip):
    model.train()
    
    epoch_loss = 0
    batch_size = 128
    data_loader = DataLoader(train_set, batch_size=batch_size, shuffle=True)
    trange = tqdm(enumerate(data_loader), total=len(data_loader),desc='Train')
    for step,batch in trange:
        # print(step)
        src = batch[0]
        trg = batch[1]
        src = src.to(device)
        trg = trg.to(device)       
        optimizer.zero_grad()
        # print(trg.size())
        output,output_idx = model(src, trg)
        
        #trg = [trg sent len, batch size]
        #output = [trg sent len, batch size, output dim]
        
        output = output[:,1:].reshape(-1, output.shape[-1])
        trg = trg[:,1:].reshape(-1)
        
        #trg = [(trg sent len - 1) * batch size]
        #output = [(trg sent len - 1) * batch size, output dim]
        
        loss = criterion(output, trg)
        
        loss.backward()
        
        torch.nn.utils.clip_grad_norm_(model.parameters(), clip)
        
        optimizer.step()
        
        epoch_loss += loss.item()
    return epoch_loss / len(data_loader)


In [17]:
def decode_string(output,dataset):
    results = []
    trg = []
    correct = 0
    for i,seq in enumerate(output):
        results.append('<sos> '+dataset.vocab.indices_to_sequence(seq))
        trg.append(' '.join(dataset.raw_data[i]))
        if results[-1] == trg[-1]:
            correct+=1
    print('-----exm-----')
    print(results[:10])
    print(trg[:10])
    return results, correct/len(output)

def evaluate(model,criterion,dataset):
    prediction = []
    model.eval()
    
    epoch_loss = 0
    data_loader = DataLoader(dataset, batch_size=128, shuffle=False)
    trange = tqdm(enumerate(data_loader), total=len(data_loader),desc='valid')
    for step, batch in trange:
        src = batch[0]
        trg = batch[1]
        src = src.to(device)
        trg = trg.to(device)
        with torch.no_grad():
            output,output_idx = model(src, trg, 0) #turn off teacher forcing
            prediction.append(output_idx.to('cpu'))

            #trg = [trg sent len, batch size]
            #output = [trg sent len, batch size, output dim]

            output = output[:,1:].reshape(-1, output.shape[-1])
            trg = trg[:,1:].reshape(-1)

            #trg = [(trg sent len - 1) * batch size]
            #output = [(trg sent len - 1) * batch size, output dim]

            loss = criterion(output, trg)

            epoch_loss += loss.item()
    prediction = torch.cat(prediction).detach().numpy().astype(int)
    prediction,acc = decode_string(prediction,dataset)
    return epoch_loss / len(data_loader),prediction,acc

In [18]:
def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

In [19]:
N_EPOCHS = 10
CLIP = 1

best_valid_loss = float('inf')
best_valid_acc = 0
history = {'train':[],'val':[]}

for epoch in range(N_EPOCHS):
    
    start_time = time.time()
    
    train_loss = train(model, optimizer, criterion, CLIP)
    train_loss_2,predictions,acc = evaluate(model, criterion,train_set)
    history['train'].append({'loss':train_loss,'acc':acc})
    valid_loss,valid_predict,valid_acc = evaluate(model, criterion,test_set)
    end_time = time.time()
    
    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model, 'model_'+str(int(valid_acc*10000))+'.pkl') 
        torch.save(model.state_dict(), 'param_'+str(int(valid_acc*10000))+'.pt')
    history['val'].append({'loss':valid_loss,'acc':valid_acc})
    print(f'Epoch: {epoch+1:02} | Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train PPL: {math.exp(train_loss):7.3f} | Train acc: {acc:.3f}')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. PPL: {math.exp(valid_loss):7.3f} | Train acc: {valid_acc:.3f}')

valid:   1%|▏         | 2/160 [00:00<00:13, 11.53it/s]

-----exm-----
['<sos> tom seemed annoyed <eos>', '<sos> i was celebrating <eos>', '<sos> please behave windsurfing <eos>', "<sos> who 's overreacting <eos>", '<sos> i like australia <eos>', '<sos> conditions all devastating <eos>', '<sos> tom fired me <eos>', "<sos> tom 's interesting <eos>", '<sos> it was marvelous <eos>', "<sos> you 're wise <eos>"]
['<sos> tom seemed annoyed <eos>', '<sos> i was enthusiastic <eos>', '<sos> please behave prudently <eos>', "<sos> who 's overreacting <eos>", '<sos> i like australia <eos>', '<sos> heed all warnings <eos>', '<sos> tom fired me <eos>', "<sos> tom 's interesting <eos>", '<sos> it was marvelous <eos>', "<sos> you 're wise <eos>"]


valid: 100%|██████████| 160/160 [00:13<00:00, 11.44it/s]
/root/anaconda3/lib/python3.6/site-packages/torch/serialization.py:256: UserWarning: Couldn't retrieve source code for container of type Seq2Seq. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
/root/anaconda3/lib/python3.6/site-packages/torch/serialization.py:256: UserWarning: Couldn't retrieve source code for container of type Encoder. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
/root/anaconda3/lib/python3.6/site-packages/torch/serialization.py:256: UserWarning: Couldn't retrieve source code for container of type Decoder. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
Train:   0%|          | 0/2991 [00:00<?, ?it/s]

-----exm-----
['<sos> you know me <eos>', "<sos> i 'll leave <eos>", "<sos> we 'll see <eos>", '<sos> they were right <eos>', "<sos> do n't speak <eos>", '<sos> they know something <eos>', '<sos> who told tom <eos>', '<sos> is that all <eos>', '<sos> you made it <eos>', '<sos> tom is back <eos>']
['<sos> you know me <eos>', "<sos> i 'll leave <eos>", "<sos> we 'll see <eos>", '<sos> they were right <eos>', "<sos> do n't speak <eos>", '<sos> they know something <eos>', '<sos> who told tom <eos>', '<sos> is that all <eos>', '<sos> you made it <eos>', '<sos> tom is back <eos>']
Epoch: 01 | Time: 10m 49s
	Train Loss: 1.624 | Train PPL:   5.073 | Train acc: 0.742
	 Val. Loss: 2.767 |  Val. PPL:  15.910 | Train acc: 0.201


valid:   1%|▏         | 2/160 [00:00<00:13, 11.38it/s]

...

-----exm-----
['<sos> you know me <eos>', "<sos> i 'll leave <eos>", "<sos> we 'll see <eos>", '<sos> they were right <eos>', "<sos> do n't speak <eos>", '<sos> they know something <eos>', '<sos> who told tom <eos>', '<sos> is that all <eos>', '<sos> you made it <eos>', '<sos> tom is back <eos>']
['<sos> you know me <eos>', "<sos> i 'll leave <eos>", "<sos> we 'll see <eos>", '<sos> they were right <eos>', "<sos> do n't speak <eos>", '<sos> they know something <eos>', '<sos> who told tom <eos>', '<sos> is that all <eos>', '<sos> you made it <eos>', '<sos> tom is back <eos>']
Epoch: 09 | Time: 10m 53s
	Train Loss: 0.022 | Train PPL:   1.022 | Train acc: 0.962
	 Val. Loss: 1.080 |  Val. PPL:   2.945 | Train acc: 0.733


valid:   1%|▏         | 2/160 [00:00<00:13, 11.52it/s]

-----exm-----
['<sos> tom seemed annoyed <eos>', '<sos> i was enthusiastic <eos>', '<sos> please behave prudently <eos>', "<sos> who 's overreacting <eos>", '<sos> i like australia <eos>', '<sos> heed all warnings <eos>', '<sos> tom fired me <eos>', "<sos> tom 's interesting <eos>", '<sos> it was marvelous <eos>', "<sos> you 're wise <eos>"]
['<sos> tom seemed annoyed <eos>', '<sos> i was enthusiastic <eos>', '<sos> please behave prudently <eos>', "<sos> who 's overreacting <eos>", '<sos> i like australia <eos>', '<sos> heed all warnings <eos>', '<sos> tom fired me <eos>', "<sos> tom 's interesting <eos>", '<sos> it was marvelous <eos>', "<sos> you 're wise <eos>"]


valid: 100%|██████████| 160/160 [00:14<00:00, 11.39it/s]


-----exm-----
['<sos> you know me <eos>', "<sos> i 'll leave <eos>", "<sos> we 'll see <eos>", '<sos> they were right <eos>', "<sos> do n't speak <eos>", '<sos> they know something <eos>', '<sos> who told tom <eos>', '<sos> is that all <eos>', '<sos> you made it <eos>', '<sos> tom is back <eos>']
['<sos> you know me <eos>', "<sos> i 'll leave <eos>", "<sos> we 'll see <eos>", '<sos> they were right <eos>', "<sos> do n't speak <eos>", '<sos> they know something <eos>', '<sos> who told tom <eos>', '<sos> is that all <eos>', '<sos> you made it <eos>', '<sos> tom is back <eos>']
Epoch: 10 | Time: 10m 46s
	Train Loss: 0.020 | Train PPL:   1.020 | Train acc: 0.974
	 Val. Loss: 1.050 |  Val. PPL:   2.857 | Train acc: 0.737


In [20]:
import pickle
pickle.dump(history,open('history1.pkl','wb'))
history

{'train': [{'loss': 1.623956356866429, 'acc': 0.7423253262275168},
  {'loss': 0.22696194058425528, 'acc': 0.9092467050694254},
  {'loss': 0.09705644825847701, 'acc': 0.9500006531081416},
  {'loss': 0.05352617025005453, 'acc': 0.9614274331543817},
  {'loss': 0.035866831190166704, 'acc': 0.9699909871076453},
  {'loss': 0.028789088557670792, 'acc': 0.9700536854892433},
  {'loss': 0.025988047706808327, 'acc': 0.9666470734224173},
  {'loss': 0.022813581867549546, 'acc': 0.9734602975560693},
  {'loss': 0.021725931917670624, 'acc': 0.9621301775147929},
  {'loss': 0.019985367443109886, 'acc': 0.9740507073161174}],
 'val': [{'loss': 2.766957180562895, 'acc': 0.20122399020807832},
  {'loss': 2.2696548412000994, 'acc': 0.35290085679314565},
  {'loss': 1.4064702002659033, 'acc': 0.5935862913096696},
  {'loss': 1.1270683588234534, 'acc': 0.7147613219094248},
  {'loss': 1.0957045394850866, 'acc': 0.7210771113831089},
  {'loss': 1.0223493266442347, 'acc': 0.7279314565483476},
  {'loss': 1.11381260499

In [20]:
import numpy as np
a=np.array([[[1,1],[2,1],[3,1]],[[4,1],[5,1],[6,1]],[[7,1],[8,1],[9,1]]])
a

array([[[1, 1],
        [2, 1],
        [3, 1]],

       [[4, 1],
        [5, 1],
        [6, 1]],

       [[7, 1],
        [8, 1],
        [9, 1]]])

In [30]:
print(OOV)
print(OOV_list)

0
[]


In [50]:
a[:,2]

array([3, 6, 9])

In [66]:
m = nn.Dropout(p=0.2)
input = torch.randn(10,1)
output = m(input)
output

tensor([[-1.1133],
        [ 0.0000],
        [ 1.9238],
        [ 1.8364],
        [ 0.4400],
        [-1.7066],
        [-2.8024],
        [-2.0669],
        [-0.0000],
        [ 0.8218]])

In [28]:
outputs = torch.zeros(3, 2, 4)
a = torch.ones(3,4)

In [29]:
outputs[:,0,:] = a

In [30]:
outputs

tensor([[[1., 1., 1., 1.],
         [0., 0., 0., 0.]],

        [[1., 1., 1., 1.],
         [0., 0., 0., 0.]],

        [[1., 1., 1., 1.],
         [0., 0., 0., 0.]]])